# Using VGG16 for our Face detection

### Loading the VGG16 Model

In [1]:
from keras.applications import vgg16

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the vgg16 model without the top or FC layers
# Using include_top = False, we disable the dense layers or Fully Connected layers 
# These are the layers that classify using the patterns found in crp layers

#Loads the VGG16 model 
model = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze all the Layers
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [2]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Let's make a function that returns our FC Head

In [3]:
def addTopModel(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

### Let's add our FC Head back onto our model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


num_classes = 2
FC_Head = addTopModel(model, num_classes)
model = Model(inputs = model.input, outputs = FC_Head)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Creating the Dataset

In [ ]:
import time
import cv2
cap = cv2.VideoCapture(0)

In [ ]:
# Collect 1000 samples of your face from webcam input
count=0
while True:
    ret, face = cap.read()
    count += 1
    
    # Save file in specified directory with unique name
    file_name_path = 'C:\Work\Mlops\Computer Vision\Datasets\Face Data\Sunny_' + str(count) + '.jpg'
    cv2.imwrite(file_name_path, face)

    # Put count on images and display live count
    cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Face', face)
    time.sleep(0.1)
        
    if cv2.waitKey(1) == 13 or count == 1000: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Loading our  Dataset

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Work/Mlops/Computer Vision/Datasets/Face Data/Train/'
validation_data_dir = 'C:/Work/Mlops/Computer Vision/Datasets/Face Data/Test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 1700 images belonging to 2 classes.
Found 310 images belonging to 2 classes.


### Training out Model
- Note we're using checkpointing and early stopping

In [6]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("Face_vgg16model_2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1700
nb_validation_samples = 310

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
106/106 [==============================] - 711s 7s/step - loss: 0.3253 - accuracy: 0.9104 - val_loss: 1.2306e-04 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00012, saving model to Face_vgg16model_2.h5
Epoch 2/5
106/106 [==============================] - 702s 7s/step - loss: 0.1021 - accuracy: 0.9814 - val_loss: 1.1567e-05 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.00012 to 0.00001, saving model to Face_vgg16model_2.h5
Epoch 3/5
106/106 [==============================] - 468s 4s/step - loss: 0.1645 - accuracy: 0.9913 - val_loss: 6.3159e-05 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.00001
Epoch 4/5
106/106 [==============================] - 433s 4s/step - loss: 0.0581 - accuracy: 0.9916 - val_loss: 8.1956e-08 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.00001 to 0.00000, saving model to Face_vgg16model_2.h5
Epoch 5/5
106/106 [==============================] - 432s 4s/step - loss: 0.0528 - acc

### Loading our classifer


In [8]:
from keras.models import load_model

classifier = load_model('Face_vgg16model_2.h5')

### Testing our classifer on some test images

In [9]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

Faces = {"[0]": "Sunny ", 
         "[1]": "Sumeet" }

Faces_n = {"n0": "Sunny ", 
           "n1": "Sumeet"}

def draw_test(name, pred, im):
    face = Faces[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + Faces_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("C:/Work/Mlops/Computer Vision/Datasets/Face Data/Test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Sumeet
Class - Sunny 
Class - Sunny 
Class - Sunny 
Class - Sunny 
Class - Sunny 
Class - Sunny 
Class - Sumeet
Class - Sumeet
Class - Sunny 
